This Model did extremely poorly. Only got a score of 0.56675

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as TSSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

In [2]:
X = pd.read_csv("./Train_X.csv")
scaler = StandardScaler()
data = pd.read_csv("./train.csv")
y = data.iloc[:,-1]
X_train,X_cv,y_train,y_cv = TSSplit(X,y)

In [3]:
X_train.Age = X_train.Age.fillna(X_train.Age.mean())
X_train = X_train.fillna(0)
X_train.isnull().sum()


Earth            0
Europa           0
Mars             0
55 Cancri e      0
PSO J318.5-22    0
TRAPPIST-1e      0
CryoSleep        0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
dtype: int64

In [20]:
X_train = scaler.fit_transform(X_train)
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [5]:
X_cv.Age = X_cv.Age.fillna(X_cv.Age.mean())
X_cv = X_cv.fillna(0)
X_cv.isnull().sum()

Earth            0
Europa           0
Mars             0
55 Cancri e      0
PSO J318.5-22    0
TRAPPIST-1e      0
CryoSleep        0
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
dtype: int64

In [6]:
X_cv = scaler.transform(X_cv)
predictions = knn.predict(X_cv) 

In [38]:
predictions

array([False,  True, False, ..., False, False,  True])

In [7]:
from sklearn.metrics import accuracy_score
accuracy_score(y_cv,predictions)

0.7589696412143514

In [8]:
from datetime import datetime
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [15]:
a = range(1,50,1)
a

range(1, 50)

In [26]:
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
print(param_grid)

{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]}


In [29]:
from sklearn.model_selection import GridSearchCV
start_time = timer(None)
grid = GridSearchCV(estimator=knn, param_grid=param_grid, scoring='roc_auc', n_jobs=-1, cv=4, verbose=3 )
grid.fit(X_train,y_train)
timer(start_time)
# print('\n All results:')
# print(grid.cv_results_)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)
# results = pd.DataFrame(grid.cv_results_)

Fitting 4 folds for each of 30 candidates, totalling 120 fits

 Time taken: 0 hours 0 minutes and 3.32 seconds.

 Best estimator:
KNeighborsClassifier(n_neighbors=30)

 Best score:
0.7114761392387545

 Best parameters:
{'n_neighbors': 30}


In [36]:
X_test = pd.read_csv("./Test_X.csv")
X_test.Age = X_test.Age.fillna(X_test.Age.mean())
X_test = X_test.fillna(0)
X_test = scaler.transform(X_test)

C:\Users\abhir\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


array([[ 1.00000000e+00, -1.52593930e-17,  6.10375720e-17, ...,
         1.08995664e-17,  3.81484825e-17,  2.50690028e-17],
       [ 1.00000000e+00, -1.52593930e-17,  6.10375720e-17, ...,
         1.08995664e-17,  2.82300000e+03,  2.50690028e-17],
       [-4.79580923e-17,  1.00000000e+00,  6.10375720e-17, ...,
         1.08995664e-17,  3.81484825e-17,  2.50690028e-17],
       ...,
       [-4.79580923e-17, -1.52593930e-17,  1.00000000e+00, ...,
         1.08995664e-17,  3.81484825e-17,  2.50690028e-17],
       [-4.79580923e-17,  1.00000000e+00,  6.10375720e-17, ...,
         1.08995664e-17,  3.81484825e-17,  5.23000000e+02],
       [ 1.00000000e+00, -1.52593930e-17,  6.10375720e-17, ...,
         1.08995664e-17,  3.81484825e-17,  2.50690028e-17]])

In [39]:
test_predict = grid.best_estimator_.predict(X_test)
test_predict

array([False, False, False, ..., False,  True, False])

In [40]:
test_data = pd.read_csv("./test.csv")

In [41]:
results_df = pd.DataFrame(data={'PassengerId':test_data['PassengerId'], 'Transported':test_predict})
results_df

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,False
4273,9269_01,True
4274,9271_01,False
4275,9273_01,True


In [42]:
results_df.to_csv('./submission_2.csv', index=False)